# Исследование алгоритма BoVW для нашей задачи

In [1]:
import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt
import yaml

with open('config.yaml', 'r') as config:
    cfg = yaml.safe_load(config)

In [1]:
from BoVW.BoVW import BoVW
from dataset.get_pictures import DatasetOperations
from CustomDescriptors.abstract.abstract import ABSDescriptor
from CustomDescriptors.AkazeDescriptor.AKAZE import AKAZE
from CustomDescriptors.OrbDescriptor.ORB import ORB
from CustomDescriptors.ResnetDescriptor.Resnet import Resnet
from CustomDescriptors.SiftDescriptor.SIFT import SIFT
from CustomDescriptors.FaceDescriptor.FACE import FACE
from helper.helper import Research, Multiprocessor

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans, MiniBatchKMeans, SpectralBiclustering, Birch

In [2]:
DatasetOperations.clear()

In [7]:
Research.add_train_dataset(percentage=20, scale=True, resolution="low")

скопировано 20 картинок в train


In [6]:
Research.safe(
    "test",
    Research.test(
        Research.train(
            clf=BaggingClassifier(),
            cluster=Birch(
                n_clusters=20
                ),
            descriptor=FACE(
                predictor_path=cfg['Victor']['FACE']
                ),
            scale=False,
            number_words=20
        )
    )
)

KeyboardInterrupt: 

In [9]:
clfs = {
    "RandomForest": RandomForestClassifier(),
    "Bagging": BaggingClassifier(),
    "GradientBoosting": GradientBoostingClassifier(),
    "SVM": SVC(),
    "LogisticRegression": LogisticRegression(),
}

clusters = {
    "KMeans": KMeans(n_clusters=200),
    "MiniBatchKMeans": MiniBatchKMeans(n_clusters=200),
    "Birch": Birch(n_clusters=200),
    "SpectralBiclustering": SpectralBiclustering(n_clusters=200)
}

descriptors = {
    "ORB": ORB(),
    "AKAZE": AKAZE(),
    "Resnet": Resnet(),
    "SIFT" : SIFT(cfg['Victor']['Dataset'])
}

In [4]:
for name_clf, clf in clfs.items():
    for name_cluster, cluster in clusters.items():
        for name_descriptor, descriptor in descriptors.items():
            Research.safe(
                f"{name_clf}_{name_cluster}_{name_descriptor}",
                Research.test(
                    Research.train(
                        clf=clf,
                        descriptor=descriptor,
                        cluster=cluster,
                        scale=False,
                        number_words=200
                    )
                )
                
            )

NameError: name 'clfs' is not defined